In [60]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
DB_CONFIG = {
    'user': 'postgres',
    'password': 'postgre',
    'host': 'localhost',
    'port': '5432',
    'database': 'sales_db'
}


RAW_DIR = './Sales_details'
ARCHIVE_DIR = './data/processed'
os.makedirs(ARCHIVE_DIR, exist_ok=True)


engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

In [5]:
try:
    with engine.connect() as conn:
        print("Connection Successful !")
except Exception as e:
    print(f" Error connecting : {str(e)}")
    raise

Connection Successful !


In [7]:
query = """
SELECT "Order_ID", "Customer_ID", "Customer_Type", "Product", "Category", 
       "Unit_Price", "Quantity", "Discount", "Total_Price", "Region", 
       "Sales_Order_Date"
FROM sales
WHERE "Sales_Order_Date" BETWEEN '2023-01-01' AND '2024-12-31'
"""
data = pd.read_sql(query, engine)

In [9]:
data['Sales_Order_Date'] = pd.to_datetime(data['Sales_Order_Date'])

In [21]:
latest_date = data['Sales_Order_Date'].max()

rfm_all = data.groupby(['Customer_ID', 'Customer_Type']).agg({
    'Sales_Order_Date': lambda x: (latest_date - x.max()).days,
    'Order_ID': 'nunique',
    'Total_Price': 'sum',
    'Region': lambda x: x.mode()[0],  # most frequent region
}).reset_index()

In [22]:
rfm_all.columns = ['Customer_ID', 'Customer_Type', 'Recency', 'Frequency', 'Monetary', 'Region']
rfm_all['Monetary'] = rfm_all['Monetary'].replace([np.inf, -np.inf], 0).fillna(0)
rfm_all['Churn_Flag'] = rfm_all['Recency'].apply(lambda x: 1 if x > 90 else 0)

In [39]:
# AOV
rfm_all['AOV'] = rfm_all['Monetary'] / rfm_all['Frequency']


In [41]:
avg_discount = data.groupby(['Customer_ID', 'Customer_Type'])['Discount'].mean().reset_index()
avg_discount.columns = ['Customer_ID', 'Customer_Type', 'Avg_Discount']
rfm_all = pd.merge(rfm_all, avg_discount, on=['Customer_ID', 'Customer_Type'], how='left')

In [43]:
product_variety = data.groupby(['Customer_ID', 'Customer_Type'])['Product'].nunique().reset_index()
product_variety.columns = ['Customer_ID', 'Customer_Type', 'Product_Variety']
rfm_all = pd.merge(rfm_all, product_variety, on=['Customer_ID', 'Customer_Type'], how='left')

In [45]:
top_category = data.groupby(['Customer_ID', 'Customer_Type'])['Category'].agg(lambda x: x.mode()[0]).reset_index()
top_category.columns = ['Customer_ID', 'Customer_Type', 'Top_Category']
rfm_all = pd.merge(rfm_all, top_category, on=['Customer_ID', 'Customer_Type'], how='left')

In [47]:
purchase_interval = data.sort_values('Sales_Order_Date') \
    .groupby(['Customer_ID', 'Customer_Type'])['Sales_Order_Date'] \
    .apply(lambda x: x.diff().dt.days.mean()).reset_index(name='Avg_Purchase_Interval')
rfm_all = pd.merge(rfm_all, purchase_interval, on=['Customer_ID', 'Customer_Type'], how='left')

In [50]:
final_segments = []
plot_dir = "./outputs/segmentation_plots"
os.makedirs(plot_dir, exist_ok=True)

In [92]:

print("Shape of rfm_all:", rfm_all.shape)
print("\nColumns in rfm_all:")
for col in rfm_all.columns:
    print(f"- {col}")


region_cols = [col for col in rfm_all.columns if col.startswith('Region_')]
print("\nRegion columns:", region_cols)


if region_cols:
    print("\nSample of Region columns (first 5 rows):")
    print(rfm_all[region_cols].head())
    
    
    print("\nData types of Region columns:")
    print(rfm_all[region_cols].dtypes)
    
    
    print("\nMissing values in Region columns:")
    print(rfm_all[region_cols].isna().sum())

Shape of rfm_all: (10000, 12)

Columns in rfm_all:
- Customer_ID
- Customer_Type
- Recency
- Frequency
- Monetary
- Region
- Churn_Flag
- AOV
- Avg_Discount
- Product_Variety
- Top_Category
- Avg_Purchase_Interval

Region columns: []


In [94]:

rfm_clean = rfm_all.copy()


region_cols = [col for col in rfm_clean.columns if col.startswith('Region_')]
if region_cols:
    
    rfm_clean['Region'] = None
    
    
    for col in region_cols:
        region_name = col.replace('Region_', '')
        # Check if column is boolean or numeric
        if rfm_clean[col].dtype == bool:
            rfm_clean.loc[rfm_clean[col] == True, 'Region'] = region_name
        else:
            rfm_clean.loc[rfm_clean[col] == 1, 'Region'] = region_name
    
    
    print("\nRegion column value counts:")
    print(rfm_clean['Region'].value_counts())
    print("Null regions:", rfm_clean['Region'].isna().sum())
    
    # Now drop the one-hot columns
    rfm_clean = rfm_clean.drop(columns=region_cols)


cat_cols = [col for col in rfm_clean.columns if col.startswith('Top_Category_')]
if cat_cols:
   
    rfm_clean['Top_Category'] = None
    
    
    for col in cat_cols:
        cat_name = col.replace('Top_Category_', '')
        # Check if column is boolean or numeric
        if rfm_clean[col].dtype == bool:
            rfm_clean.loc[rfm_clean[col] == True, 'Top_Category'] = cat_name
        else:
            rfm_clean.loc[rfm_clean[col] == 1, 'Top_Category'] = cat_name
    
 
    print("\nTop_Category column value counts:")
    print(rfm_clean['Top_Category'].value_counts())
    print("Null categories:", rfm_clean['Top_Category'].isna().sum())
    
    
    rfm_clean = rfm_clean.drop(columns=cat_cols)


print("\nFinal rfm_clean shape:", rfm_clean.shape)
print("Final columns:", rfm_clean.columns.tolist())


rfm_all = rfm_clean


Final rfm_clean shape: (10000, 12)
Final columns: ['Customer_ID', 'Customer_Type', 'Recency', 'Frequency', 'Monetary', 'Region', 'Churn_Flag', 'AOV', 'Avg_Discount', 'Product_Variety', 'Top_Category', 'Avg_Purchase_Interval']


In [102]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# lists to store final segments
final_segments = []
all_profiles = None


for ctype in rfm_all['Customer_Type'].unique():
    print(f"\n🔍 Segmenting: {ctype}")
    df = rfm_all[rfm_all['Customer_Type'] == ctype].copy()
    
    # numeric features for clustering
    numeric_features = [
        'Recency', 'Frequency', 'Monetary', 'AOV', 'Avg_Discount',
        'Product_Variety', 'Avg_Purchase_Interval'
    ]
    
    #only include columns that exist
    numeric_features = [col for col in numeric_features if col in df.columns]
    
    #dataframe for clustering
    X = df[numeric_features].copy()
    
    
    if 'Region' in df.columns and df['Region'].notna().any():
        encoder = LabelEncoder()
        X['Region_encoded'] = encoder.fit_transform(df['Region'].fillna('Unknown'))
        print(f"Region values mapped to: {dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))}")
    
    if 'Top_Category' in df.columns and df['Top_Category'].notna().any():
        encoder = LabelEncoder()
        X['Top_Category_encoded'] = encoder.fit_transform(df['Top_Category'].fillna('Unknown'))
    
    # all features to use for clustering
    all_features = numeric_features.copy()
    if 'Region_encoded' in X.columns:
        all_features.append('Region_encoded')
    if 'Top_Category_encoded' in X.columns:
        all_features.append('Top_Category_encoded')
    
    # drop rows with any missing values in clustering features
    mask = X[all_features].notna().all(axis=1)
    X_filtered = X[mask].copy()
    df_filtered = df[mask].copy()  
    
    if len(X_filtered) < 2:
        print(f"WARNING: Not enough data points for {ctype} after filtering NA values. Skipping.")
        continue
    
    # Scale features 
    X_scaled = StandardScaler().fit_transform(X_filtered[all_features])
    
    # Elbow Method
    inertias = []
    max_k = min(7, len(X_filtered))
    for k in range(2, max_k):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
    
    plt.figure()
    plt.plot(range(2, max_k), inertias, marker='o')
    plt.title(f"Elbow Plot - {ctype}")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Inertia")
    plt.grid(True)
    plt.savefig(f"{plot_dir}/elbow_{ctype}.png")
    plt.close()
    
    # Clustering
    optimal_k = min(3, len(X_filtered) - 1)  # Ensure we don't try to create more clusters than data points
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    cluster_assignments = kmeans.fit_predict(X_scaled)
    
    
    df_clustered = df_filtered.copy()
    df_clustered['Cluster'] = cluster_assignments
    
    # Debug info
    print(f"Cluster distribution for {ctype}: {df_clustered['Cluster'].value_counts().to_dict()}")
    
    # cluster profiles using original categorical values
    cluster_profiles = df_clustered.groupby('Cluster')[numeric_features].median().reset_index()
    
    #  get most common value per cluster
    if 'Region' in df.columns and df['Region'].notna().any():
        region_mode = df_clustered.groupby('Cluster')['Region'].agg(
            lambda x: x.mode()[0] if not x.mode().empty else None
        ).reset_index()
        cluster_profiles = cluster_profiles.merge(region_mode, on='Cluster')
    
    if 'Top_Category' in df.columns and df['Top_Category'].notna().any():
        category_mode = df_clustered.groupby('Cluster')['Top_Category'].agg(
            lambda x: x.mode()[0] if not x.mode().empty else None
        ).reset_index()
        cluster_profiles = cluster_profiles.merge(category_mode, on='Cluster')
    
    cluster_profiles['Customer_Type'] = ctype
    
   
    if all_profiles is None:
        all_profiles = cluster_profiles
    else:
        all_profiles = pd.concat([all_profiles, cluster_profiles], ignore_index=True)
    
   
    df_clustered['Segment_Label'] = df_clustered['Cluster'].map(cluster_labels)
    
    # PCA 
    pca = PCA(n_components=2)
    components = pca.fit_transform(X_scaled)
    df_clustered['PCA1'], df_clustered['PCA2'] = components[:, 0], components[:, 1]
    
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df_clustered, x='PCA1', y='PCA2', hue='Cluster', palette='Set2', s=60)
    plt.title(f"PCA Cluster Visualization - {ctype}")
    plt.legend(title="Cluster")
    plt.savefig(f"{plot_dir}/pca_{ctype}.png")
    plt.close()
    
    final_segments.append(df_clustered)

# Combine all segmented data
if final_segments:
    final_df = pd.concat(final_segments, ignore_index=True)
    print(f"Final segmentation complete with {len(final_df)} records")
else:
    print("No segments were created")


🔍 Segmenting: B2C
Region values mapped to: {'Baden-Württemberg': 0, 'Bayern': 1, 'Berlin': 2, 'Brandenburg': 3, 'Bremen': 4, 'Hamburg': 5, 'Hessen': 6, 'Mecklenburg-Vorpommern': 7, 'Niedersachsen': 8, 'Nordrhein-Westfalen': 9, 'Rheinland-Pfalz': 10, 'Saarland': 11, 'Sachsen': 12, 'Sachsen-Anhalt': 13, 'Schleswig-Holstein': 14, 'Thüringen': 15}
Cluster distribution for B2C: {1: 3205, 0: 3172, 2: 60}

🔍 Segmenting: B2B
Region values mapped to: {'Baden-Württemberg': 0, 'Bayern': 1, 'Berlin': 2, 'Brandenburg': 3, 'Bremen': 4, 'Hamburg': 5, 'Hessen': 6, 'Mecklenburg-Vorpommern': 7, 'Niedersachsen': 8, 'Nordrhein-Westfalen': 9, 'Rheinland-Pfalz': 10, 'Saarland': 11, 'Sachsen': 12, 'Sachsen-Anhalt': 13, 'Schleswig-Holstein': 14, 'Thüringen': 15}
Cluster distribution for B2B: {0: 1250, 2: 1164, 1: 1149}
Final segmentation complete with 10000 records


In [107]:
result = pd.concat(final_segments, ignore_index=True)
result.to_csv('./data/processed/customer_segments_rich.csv', index=False)

In [109]:
all_profiles.to_csv('./data/processed/segment_profiles.csv', index=False)


In [111]:
result.to_sql("customer_segments_RFM", engine, if_exists="replace", index=False)

1000

In [113]:
all_profiles.to_sql("segment_profiles", engine, if_exists="replace", index=False)

6